---
## İnceleyeceğimiz kodu aşağıya bırakıyorum:

* Bu kod referans niteliğinde olacak ve ghost conv'u bu kod üzerinden detaylandıracağız.

----

In [1]:
import torch 
import torch.nn as nn

In [3]:
class GhostConv(nn.Module):
    """
    Basit Ghost Convolution implementasyonu.

    Adımlar:
    - primary conv: C_in -> C_int  (daha az kanal, pahalı kısım)
    - cheap operation (depthwise conv): C_int -> C_ghost
    - Concat primary ve ghost, sonra C_out'e kırp
    """
    def __init__(self, c_in, c_out, kernel_size=1, ratio=2, dw_kernel_size=3,
                 stride=1, padding=0):
        super().__init__()
        self.c_out = c_out
        self.ratio = ratio

        # Dahili kanal sayısı: kaç tane "gerçek" feature üreteceğiz?
        c_int = int(round(c_out / ratio))
        self.c_int = c_int
        c_ghost = c_out - c_int

        # 1) Primary conv (pahalı kısım)
        self.primary_conv = nn.Sequential(
            nn.Conv2d(
                in_channels=c_in,
                out_channels=c_int,
                kernel_size=kernel_size,
                stride=stride,
                padding=padding,
                bias=False
            ),
            nn.BatchNorm2d(c_int),
            nn.ReLU(inplace=True)
        )

        # 2) Cheap operation (ghost feature üretimi) - depthwise conv
        if c_ghost > 0:
            self.cheap_conv = nn.Sequential(
                nn.Conv2d(
                    in_channels=c_int,
                    out_channels=c_ghost,
                    kernel_size=dw_kernel_size,
                    stride=1,
                    padding=dw_kernel_size // 2,
                    groups=c_int,   # depthwise
                    bias=False
                ),
                nn.BatchNorm2d(c_ghost),
                nn.ReLU(inplace=True)
            )
        else:
            self.cheap_conv = None

    def forward(self, x):
        # 1) Temel feature'ları üret
        x_primary = self.primary_conv(x)   # (N, c_int, H, W)

        # 2) Ghost feature'ları üret
        if self.cheap_conv is not None:
            x_ghost = self.cheap_conv(x_primary)  # (N, c_ghost, H, W)
            out = torch.cat([x_primary, x_ghost], dim=1)  # (N, c_int + c_ghost, H, W)
        else:
            out = x_primary

        # 3) Fazla kanal varsa kırp (güvenlik için)
        if out.size(1) > self.c_out:
            out = out[:, :self.c_out, :, :]

        return out



# 1. Constructor parametreleri: dış API
```python
def __init__(self, c_in, c_out, kernel_size=1, ratio=2, dw_kernel_size=3,
             stride=1, padding=0):
```


Burada GhostConv’u dışarıdan şu şekilde kullanıyorsun:

* c_in: giriş kanal sayısı (in_channels)

* c_out: istediğin toplam çıkış kanalı (out_channels, normal conv gibi)

* kernel_size: primary conv için kernel (genelde 1, bazen 3)

* ratio: C_out’un ne kadarını primary conv ile, ne kadarını ucuz ops ile üreteceğini kontrol eden oran

* ratio=2 → C_int ≈ C_out / 2 kadarını pahalı conv üretir, geri kalanı ucuzdan gelir

* dw_kernel_size: ucuz depthwise conv için kernel (genelde 3)

* stride: primary conv’un stride’ı (uzamsal downsample gerekiyorsa buradan)

* padding: primary conv için padding

----

# 2. İç kanal hesapları: C_int ve C_ghost

```python
self.c_out = c_out
self.ratio = ratio

c_int = int(round(c_out / ratio))
self.c_int = c_int
c_ghost = c_out - c_int
```
Burada yaptığımız:

* c_out sabit: sonunda eline geçmesini istediğin kanal sayısı

* ratio: “kaçta kaçı gerçek, kaçta kaçı ghost?”

* c_int = round(c_out / ratio)

**Örnek: c_out=64, ratio=2 → c_int = 32**

* c_ghost = c_out - c_int

**Örnek: 64 - 32 = 32 → 32 kanal ghost’tan gelecek**

Yani:

* Primary conv: C_in → C_int (pahalı)

* Cheap ops: C_int → C_ghost (ucuz)

* Sonunda concat: C_int + C_ghost = C_out

---

# 3. Primary Conv bloğu (pahalı kısım)

```python
self.primary_conv = nn.Sequential(
    nn.Conv2d(
        in_channels=c_in,
        out_channels=c_int,
        kernel_size=kernel_size,
        stride=stride,
        padding=padding,
        bias=False
    ),
    nn.BatchNorm2d(c_int),
    nn.ReLU(inplace=True)
)
```
Burada bildiğin klasik conv bloğu:

* Conv2d(c_in → c_int):

*Normal conv, tüm ağırlık burası*

*1×1 ise sadece kanal karıştırıyor*

*3×3 ise hem spatial hem kanal karışımı*

* BatchNorm2d(c_int) → öğrenmeyi stabilize ediyor

* ReLU → nonlineerlik

Şekiller:

**Girdi: (N, c_in, H, W)**

**Çıktı: (N, c_int, H', W')**

* *H',W' stride/padding’e göre (genelde H,W korunuyor, stride=1 ise)*

Bu kısım “Real feature” üreticisi.

---

# 4. Cheap Conv bloğu (ghost feature üretimi)
```python 
if c_ghost > 0:
    self.cheap_conv = nn.Sequential(
        nn.Conv2d(
            in_channels=c_int,
            out_channels=c_ghost,
            kernel_size=dw_kernel_size,
            stride=1,
            padding=dw_kernel_size // 2,
            groups=c_int,   # depthwise
            bias=False
        ),
        nn.BatchNorm2d(c_ghost),
        nn.ReLU(inplace=True)
    )
else:
    self.cheap_conv = None
```
Burada kritik şeyler:

* Girdi: x_primary (şekli (N, c_int, H', W'))

* in_channels = c_int

* out_channels = c_ghost

* groups = c_int → depthwise conv

**Her giriş kanalı kendi filtresiyle işleniyor**

* dw_kernel_size genelde 3, padding ile spatial boyut korunuyor

Bu katman:

* Daha az maliyetle yeni feature map’ler üretiyor,

* GhostNet mantığı: “bunlar primary feature’ların varyasyonları”.

**if c_ghost > 0 kontrolü:**

* Teorik olarak ratio öyle bir değer olabilir ki c_ghost 0 çıkar.

* O durumda cheap_conv yapmaya gerek yok, sadece primary yeter.
---

# 5. forward: akış mantığı
### 5.1. Primary feature’lar
```python
x_primary = self.primary_conv(x)   # (N, c_int, H, W)
```

* x: (N, c_in, H, W)

* x_primary: (N, c_int, H', W')

Burada tüm pahalı iş yapıldı.

### 5.2. Ghost feature’lar ve concat
```python
if self.cheap_conv is not None:
    x_ghost = self.cheap_conv(x_primary)  # (N, c_ghost, H, W)
    out = torch.cat([x_primary, x_ghost], dim=1)  # (N, c_int + c_ghost, H, W)
else:
    out = x_primary
```


cheap_conv varsa:

* x_ghost: (N, c_ghost, H', W')

Concat: kanal boyutunda birleştiriyoruz:

- (N, c_int, H', W') + (N, c_ghost, H', W')

- out: (N, c_int + c_ghost, H', W')

* Ama c_int + c_ghost = c_out olacak şekilde seçilmiş durumda zaten (ratio’dan geliyor).

### 5.3. Güvenlik amaçlı kırpma
```python
if out.size(1) > self.c_out:
    out = out[:, :self.c_out, :, :]
```


Neden var?

* c_int = round(c_out / ratio) dedik.

* Round işlemi bazı kombinasyonlarda c_int + c_ghost’u c_out’tan 1 fazla yapabilir (örnek: yuvarlama farkı).

* Böyle edge-case’lerde “fazla gelen 1-2 kanalı kes” demek için bu guard var.

Normal durumda:

* out.size(1) == c_out olur,

* Bu if bloğu hiçbir şey yapmaz.


# 6. Özet: GhostConv veri akışı

Bir çağrıda ne oluyor, N,C,H,W olarak:

* Örnek: c_in=32, c_out=64, ratio=2, dw_kernel_size=3, stride=1:

1.)
```python
c_int = round(64 / 2) = 32
c_ghost = 64 - 32 = 32
```

2.) Primary:

* Conv: 32 → 32

* Çıkış: x_primary: (N, 32, H, W)

3.) Cheap:

* Depthwise: 32 → 32

* Çıkış: x_ghost: (N, 32, H, W)

4.) Concat:

* out: (N, 64, H, W)

5.) Slice (gerekmiyorsa dokunmaz):

* Son: (N, 64, H, W)

**Yani dışarıdan baktığında:**

* Normal bir Conv2d(32 → 64, kernel_size=1) ile aynı çıkış shape’i veriyor,

* Ama parametreleri azaltılmış.